In [1]:
from transformer_nb import *

In [2]:
import torch.nn as nn
import torch
from torch.autograd import Variable
import json
import numpy as np
from tqdm import tqdm_notebook as tqdm
# from tqdm import tqdm
import math

In [3]:
# folder = '../pointer-generator/preprocessing-cnn-all/'
folder = 'data/'
data_name = folder+'train_seq.json'
validation_name = folder+'valid_seq.json'
testdata_name = folder+'testdata_seq.json'
vocab_name = folder+'vocab.json'

In [4]:
num_epochs = 10
save_rate = 1 #how many epochs per modelsave
continue_from = "trained/Model2" # if none, put None
continue_from = None
epsilon = 1e-10
validation_size = 5000

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
vocab = json.load(open(vocab_name, 'r'))
VOC_SIZE = len(vocab)
INPUT_MAX = 75
OUTPUT_MAX = 20
UNK = "[UNK]"
BOS = "[CLS]"
EOS = "[SEP]"
PAD = "[PAD]"

In [8]:
from dataset import make_data_generator

batch_size = 64

training_set, training_generator = make_data_generator(\
data_name, INPUT_MAX, OUTPUT_MAX, vocab[PAD], batch_size, cutoff=None, shuffle=True, num_workers=4)

validation_set, validation_generator = make_data_generator(\
validation_name, INPUT_MAX, OUTPUT_MAX, vocab[PAD], batch_size, cutoff=validation_size, shuffle=False, num_workers=4)

def data_gen_train():
    for src,tgt in training_generator:
        src = Variable(src, requires_grad=False).to(device)
        tgt = Variable(tgt, requires_grad=False).to(device)
        yield Batch(src, tgt, vocab[PAD])
def data_gen_val():
    for src,tgt in validation_generator:
        src = Variable(src, requires_grad=False).to(device)
        tgt = Variable(tgt, requires_grad=False).to(device)
        yield Batch(src, tgt, vocab[PAD])

loading json
load json done.



loading json
load json done.


In [9]:
total_train = int(math.ceil(training_set.size / batch_size))
total_valid = int(math.ceil(validation_set.size / batch_size))
print(total_train, total_valid)

58846 79


In [10]:
criterion = LabelSmoothing(size=VOC_SIZE, padding_idx=vocab[PAD], smoothing=0.0)
model = make_model(VOC_SIZE, VOC_SIZE, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1)
# model = make_model(VOC_SIZE, VOC_SIZE, N=4, d_model=256, d_ff=1024, h=8, dropout=0.1)


if continue_from == None:
    model_opt = NoamOpt(model.src_embed[0].d_model, 1, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
else:
    saved_model = torch.load(continue_from)
    model.load_state_dict(saved_model['model'])
    model_opt = saved_model['optim']

criterion.cuda()
model.cuda()

/home/george/.conda/envs/pytorch/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): Linear(in_features=512, out_features=512, bias=True)
            (2): Linear(in_features=512, out_features=512, bias=True)
            (3): Linear(in_features=512, out_features=512, bias=True)
          )
          (dropout): Dropout(p=0.1)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.1)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
      

In [11]:
# model.cuda()
# criterion.cuda()
train_history = []
valid_history = []
for epoch in range(1, num_epochs+1):
    print("Epoch", epoch)
        
    model.train()
    tl = run_epoch(data_gen_train(), model, 
              SimpleLossCompute(model.generator, criterion, model_opt), total=total_train)
    
    model.eval()
    vl = run_epoch(data_gen_val(), model, 
                    SimpleLossCompute(model.generator, criterion, None), total=total_valid)
    print("epoch train loss:", tl)
    print("epoch val loss:", vl)
    
    train_history.append(tl)
    valid_history.append(vl)
    
    try:
        torch.save({'model':model.state_dict(),
                   'optim': model_opt, 
                   'train_history': train_history,
                   'valid_history': valid_history}, 'trained/Model'+str(epoch))
    except:
        continue

Epoch 1


KeyboardInterrupt: 